In [1]:
from langchain_groq import ChatGroq

In [7]:
llm = ChatGroq(
    temperature=0, 
    groq_api_key='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', 
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [9]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-45214?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


Apply for Senior Manager, Applied Analytics

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsData & AnalyticsSenior Manager, Applied AnalyticsShanghai, China MainlandBecome a Part of the

In [11]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [13]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Senior Manager, Applied Analytics',
 'experience': '6-8 years',
 'skills': ['Data science',
  'Business operation',
  'Recap analysis',
  'Analytics skills',
  'Numeric acumen',
  'Cross-functional collaboration',
  'Problem-solving',
  'Project management',
  'Leadership'],
 'description': 'Define & develop data-driven methodology and solutions to sort out business and data process, identify key business factors and requirements, and generate business insights & resolve problems. Collaborate with business stakeholders and cross-functional teams to build trust partnerships and support stakeholders to make critical business decisions. Track & identify potential opportunities & risks in multiple daily business scenarios, create sufficient validation points for strategic options through scientific approach. Form a new team squad to work together and grow together, and be responsible for team dynamics and talent career development.'}

In [15]:
type(json_res)


dict

In [19]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://mfaiz01.github.io/MyPortfolio/
1,"Angular,.NET, SQL Server",https://mfaiz01.github.io/MyPortfolio/
2,"Vue.js, Ruby on Rails, PostgreSQL",https://mfaiz01.github.io/MyPortfolio/
3,"Python, Django, MySQL",https://mfaiz01.github.io/MyPortfolio/
4,"Java, Spring Boot, Oracle",https://mfaiz01.github.io/MyPortfolio/
5,"Flutter, Firebase, GraphQL",https://mfaiz01.github.io/MyPortfolio/
6,"WordPress, PHP, MySQL",https://mfaiz01.github.io/MyPortfolio/
7,"Magento, PHP, MySQL",https://mfaiz01.github.io/MyPortfolio/
8,"React Native, Node.js, MongoDB",https://mfaiz01.github.io/MyPortfolio/
9,"iOS, Swift, Core Data",https://mfaiz01.github.io/MyPortfolio/


In [24]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [32]:
links = collection.query(query_texts=['Expereience in Python','Expertise in React'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://mfaiz01.github.io/MyPortfolio/'},
  {'links': 'https://mfaiz01.github.io/MyPortfolio/'}],
 [{'links': 'https://mfaiz01.github.io/MyPortfolio/'},
  {'links': 'https://mfaiz01.github.io/MyPortfolio/'}]]

In [34]:
job = json_res
job['skills']

['Data science',
 'Business operation',
 'Recap analysis',
 'Analytics skills',
 'Numeric acumen',
 'Cross-functional collaboration',
 'Problem-solving',
 'Project management',
 'Leadership']

In [36]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Unlock Data-Driven Insights with AtliQ's Expertise

Dear Hiring Manager,

I came across the Senior Manager, Applied Analytics role at your esteemed organization and was impressed by the emphasis on leveraging data science to drive business growth. As a Business Development Executive at AtliQ, I'd like to introduce our company's capabilities in empowering businesses like yours to harness the power of data-driven decision-making.

AtliQ is an AI & Software Consulting company dedicated to facilitating seamless integration of business processes through automated tools. Our team of experts has a proven track record of delivering tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency.

Our expertise in data science, business operations, and analytics skills aligns perfectly with the requirements of the Senior Manager, Applied Analytics role. We've helped numerous enterprises unlock business insights, identify key factors, 

In [38]:
job

{'role': 'Senior Manager, Applied Analytics',
 'experience': '6-8 years',
 'skills': ['Data science',
  'Business operation',
  'Recap analysis',
  'Analytics skills',
  'Numeric acumen',
  'Cross-functional collaboration',
  'Problem-solving',
  'Project management',
  'Leadership'],
 'description': 'Define & develop data-driven methodology and solutions to sort out business and data process, identify key business factors and requirements, and generate business insights & resolve problems. Collaborate with business stakeholders and cross-functional teams to build trust partnerships and support stakeholders to make critical business decisions. Track & identify potential opportunities & risks in multiple daily business scenarios, create sufficient validation points for strategic options through scientific approach. Form a new team squad to work together and grow together, and be responsible for team dynamics and talent career development.'}